### **quantum eigensolver approach**

In [3]:
!pip install qiskit_nature qiskit qiskit_ibm_runtime pylatexenc pyscf qiskit_algorithms
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

  Using cached qiskit_nature-0.7.2-py3-none-any.whl.metadata (8.0 kB)
  Using cached qiskit-1.3.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached qiskit_ibm_runtime-0.34.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached pylatexenc-2.10-py3-none-any.whl
  Using cached pyscf-2.7.0-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
  Using cached qiskit_algorithms-0.3.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached h5py-3.12.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached rustworkx-0.15.1-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.9 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached stevedore-5.4.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached symengine-0.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached requests_ntlm-1.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cac

### **Molecule**

In [27]:
%%time
driver = PySCFDriver(
    # H2 (Hydrogen, 2 atoms)
    #atom="H 0 0 0; H 0 0 0.735",

#    hydronium
#    atom = "O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0; H 0.0 -0.8 0.0;",

#    H2O (Water, 3 atoms)
    atom="O 0 0 0; H 0.757 0.586 0; H -0.757 0.586 0",

#    NH3 (Ammonia, 4 atoms)
#    atom="N 0 0 0; H 0.94 0.333 0.333; H -0.94 0.333 -0.333; H 0 0.94 -0.333",

#    CH4 (Methane, 5 atoms)
#    atom="C 0 0 0; H 0.629 0.629 0.629; H -0.629 -0.629 0.629; H -0.629 0.629 -0.629; H 0.629 -0.629 -0.629",

#    LiH
#    atom= "Li 0 0 0; H 0 0 1.595",

# ammonium
#atom = "N 0 0 0; H 0.937 0 0; H -0.312 0.906 0; H -0.312 -0.453 0.785; H -0.312 -0.453 -0.785",
    

    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

CPU times: user 981 μs, sys: 0 ns, total: 981 μs
Wall time: 599 μs


### **Electronic Structure problem**

In [28]:
%%time
problem = driver.run()
print(problem)

CPU times: user 271 ms, sys: 6.83 ms, total: 277 ms
Wall time: 67 ms


### **Initial State for Eigensolver**

In [29]:
%%time
initial_state = HartreeFock(
    problem.num_spatial_orbitals,
    problem.num_particles,
    JordanWignerMapper(),
)

CPU times: user 7.55 ms, sys: 0 ns, total: 7.55 ms
Wall time: 6.69 ms


### **Ansatz for Eigensolver**

In [30]:
%%time
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    JordanWignerMapper(),
    initial_state=initial_state,
)

CPU times: user 248 ms, sys: 3.99 ms, total: 252 ms
Wall time: 251 ms


In [31]:
%%time
vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

CPU times: user 63.1 ms, sys: 3 μs, total: 63.1 ms
Wall time: 60.8 ms


<timed exec>:1: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.


In [32]:
%%time
calc = GroundStateEigensolver(JordanWignerMapper(), vqe_solver)

CPU times: user 9 μs, sys: 0 ns, total: 9 μs
Wall time: 12.2 μs


### **Eigensolver Result**

In [ ]:
%%time
result = calc.solve(problem)
print(result)